<a href="https://colab.research.google.com/github/hudada369/fy2020-repo-config/blob/master/Pytorch09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**pytorch的正则化以及偏差方差**
Droupout，Batch Normalization和正则化层
1. 正则化之 weight_decay (正则化与偏差和方差， L2 正则化）
2. 正则化之 Dropout（概念以及如何用）
3. 标准化之 Batch Normalization
4. Normalization-layers（Layer Normalization、Instance Normalization、Groupb Normalization）

误差可分解为：偏差，方差与噪声之和
1. 偏差度量了学习算法的期望预测与真实结果的偏离程度， 即刻画了学习算法本身的拟合能力
2. 方差度量了同样大小的训练集的变动所导致的学习性能的变化，即刻画了数据扰动所造成的影响
3. 噪声则表达了在当前任务上任何学习算法所能达到的期望泛化误差的下界。

L1 正则化的特点：

1. 不容易计算， 在零点连续但不可导， 需要分段求导
2. L1 模型可以将 一些权值缩小到零（稀疏）
3. 执行隐式变量选择。这意味着一些变量值对结果的影响降为 0， 就像删除它们一样
4. 其中一些预测因子对应较大的权值， 而其余的（几乎归零）
5. 由于它可以提供稀疏的解决方案， 因此通常是建模特征数量巨大时的首选模型
6. 它任意选择高度相关特征中的任何一个，并将其余特征对应的系数减少到 0**
7. L1 范数对于异常值更具提抗力

L2 正则化的特点：

1. 容易计算， 可导， 适合基于梯度的方法
2. 将一些权值缩小到接近 0
3. 相关的预测特征对应的系数值相似
4. 当特征数量巨大时， 计算量会比较大
5. 对于有相关特征存在的情况，它会包含所有这些相关的特征， 但是相关特征的权值分布取决于相关性。
6. 对异常值非常敏感
7. 相对于 L1 正则会更加准确

In [ ]:
#  L2 正则项， 在 Pytorch 中， L2 正则项又叫做 weight decay (权值衰减)
#============== step 1/5 数据 ================
def gen_data (num_data=10, x_range=(-1, 1)):
 w = 1.5
 train_x = troch.linspace (*x_range, num_data).unsqueeze_(1)
 train_y = w*train_x + torch.normal (0, 0.5, size=train_x.size ())
 test_x = torch.linspace (*x_range, num_data).unsqueeze_(1)
 test_y = w*test_x + torch.normal (0, 0.3, size=test_x.size ())
 
 return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = gen_data (x_range=(-1, 1))

#===============step 2/5 模型 ================
class MLP (nn.Module):
 def __init__(self, neural_num):
  super (MLP, self).__init__()
  self.linears = nn.Sequential (
   nn.Linear (1, neural_num),
   nn.ReLU (inplace=True),
   nn.Linear (neural_num, neural_num),
   nn.ReLU (inplace=True),
   nn.Linear (neural_num, neural_num),
   nn.ReLU (inplace=True),
   nn.Linear (neural_num, 1)
   )
 def forward (self, x):
  return self.linears (x)

# 这里建立两个模型，一个不带正则，一个带正则
net_normal = MLP (neural_num=200)
net_weight_decay = MLP (neural_num=200)

#===================step 3/5 优化器 ===================
optim_normal = torch.optim.SGD (net_normal.parameters (), lr=0.01, momentum=0.9)
optim_wdecay = torch.optim.SGD (net_weight_decay.parameters (), lr=0.01, momentum=0.9, weight_decay=1e-2)

#===================step 4/5  损失函数 ================
loss_func = torch.nn.MSELoss ()

#===================step 5/5 迭代训练 ===================
writer = SummaryWriter (comment='_test tensorboard', filename_suffix='12345678)
for epoch in range (2000):
 # forward
 pred_normal, pred_decay = net_normal (train_x), net_weight_decay (train_x)
 loss_normal, loss_wdecay = loss_func (pred_normal, train_y), loss_func (pred_wdecay, train_y)

 optim_normal.zero_grad ()
 optim_wdecay.zero_grad ()

 loss_normal.backward ()
 loss_wdecay.backward ()

 optim_normal.step ()
 optim_wdecay.step ()

 if (epoch+1) % 200 == 0:
  # 可视化
  for name, layer in net_normal.named_parameters ():
   writer.add_histogram (name+'_grad_normal', layer.grad, epoch)
   writer.add_histogram (name+'_data_normal', layer, epoch)

  for name, layer in net_weight_decay.named_parameters ():
   writer.add_histogram (name+'_grad_weight_decay', layer.grad, epoch)
   writer.add_histogram (name+'_data_weight_decay', layer, epoch)
  
  test_pred_normal, test_pred_wdecay = net_normal (test_x), net_weight_decay (test_x)

        # 绘图
        plt.scatter (train_x.data.numpy (), train_y.data.numpy (), c='blue', s=50, alpha=0.3, label='train')
        plt.scatter (test_x.data.numpy (), test_y.data.numpy (), c='red', s=50, alpha=0.3, label='test')
        plt.plot (test_x.data.numpy (), test_pred_normal.data.numpy (), 'r-', lw=3, label='no weight decay')
        plt.plot (test_x.data.numpy (), test_pred_wdecay.data.numpy (), 'b--', lw=3, label='weight decay')
        plt.text (-0.25, -1.5, 'no weight decay loss={:.6f}'.format (loss_normal.item ()), fontdict={'size': 15, 'color': 'red'})
        plt.text (-0.25, -2, 'weight decay loss={:.6f}'.format (loss_wdecay.item ()), fontdict={'size': 15, 'color': 'red'})

        plt.ylim ((-2.5, 2.5))
        plt.legend (loc='upper left')
        plt.title ("Epoch: {}".format (epoch+1))
        plt.show ()
        plt.close ()

正则化之Droupout：Dropout 叫做随机失活。就是我们给出一个概率 (随机)，让某个神经元的权重为 0 (失活)
1. 只在训练的时候开启 Dropout，而测试的时候是不用 Dropout 的
2. 上面导致的问题：数据尺度（由于部分权重的消失，导致了数据尺缩小），所以在测试的时候对权重也进行一样的数据尺度
  1. 测试的时候，所有权重都乘以 1-drop_prob， 以保证训练和测试时尺度变化一致， drop_prob 是我们的随机失活概率
  2. 在具体实现的时候：Pytorch 在实现 Dropout 的时候， 是权重乘以1/1-p的，也就是除以 1-p, 这样就不用再测试的时候权重乘以 1-p 了
3. torch.nn.Droupout(p=0.5,inplace=) p 就是被舍弃概率，也就是失活概率。

In [ ]:
# 用上面正则化 L2 的代码实例看看不用 L2，而是加上 Dropout 的效果
# ============================ step 1/5 数据 ============================
def gen_data (num_data=10, x_range=(-1, 1)):

    w = 1.5
    train_x = torch.linspace (*x_range, num_data).unsqueeze_(1)
    train_y = w*train_x + torch.normal (0, 0.5, size=train_x.size ())
    test_x = torch.linspace (*x_range, num_data).unsqueeze_(1)
    test_y = w*test_x + torch.normal (0, 0.3, size=test_x.size ())

    return train_x, train_y, test_x, test_y


train_x, train_y, test_x, test_y = gen_data (x_range=(-1, 1))


# ============================ step 2/5 模型 ============================
class MLP (nn.Module):
    def __init__(self, neural_num, d_prob=0.5):
        super (MLP, self).__init__()
        self.linears = nn.Sequential (

            nn.Linear (1, neural_num),
            nn.ReLU (inplace=True),
# 注意这里用上了 Dropout， 我们看到这个 Dropout 是接在第二个 Linear 
# 之前，Dropout 通常放在需要 Dropout 网络的前一层
            nn.Dropout (d_prob),             
            nn.Linear (neural_num, neural_num),
            nn.ReLU (inplace=True),

            nn.Dropout (d_prob),
            nn.Linear (neural_num, neural_num),
            nn.ReLU (inplace=True),
# 通常输出层的 Dropout 是不加的，这里由于数据太简单了才加上
            nn.Dropout (d_prob),  
            nn.Linear (neural_num, 1),
        )

    def forward (self, x):
        return self.linears (x)


net_prob_0 = MLP (neural_num=n_hidden, d_prob=0.)
net_prob_05 = MLP (neural_num=n_hidden, d_prob=0.5)

# ============================ step 3/5 优化器 ============================
optim_normal = torch.optim.SGD (net_prob_0.parameters (), lr=lr_init, momentum=0.9)
optim_reglar = torch.optim.SGD (net_prob_05.parameters (), lr=lr_init, momentum=0.9)

# ============================ step 4/5 损失函数 ============================
loss_func = torch.nn.MSELoss ()

# ============================ step 5/5 迭代训练 ============================

writer = SummaryWriter (comment='_test_tensorboard', filename_suffix="12345678")
for epoch in range (max_iter):

    pred_normal, pred_wdecay = net_prob_0 (train_x), net_prob_05 (train_x)
    loss_normal, loss_wdecay = loss_func (pred_normal, train_y), loss_func (pred_wdecay, train_y)

    optim_normal.zero_grad ()
    optim_reglar.zero_grad ()

    loss_normal.backward ()
    loss_wdecay.backward ()

    optim_normal.step ()
    optim_reglar.step ()

    if (epoch+1) % disp_interval == 0:

        net_prob_0.eval ()    
        # 这里要注意一下，Dropout 在训练和测试阶段不一样，这时候需要对网络设置一个状态
        net_prob_05.eval () 
        # 这个.eval () 函数表示我们的网络即将使用测试状态， 设置了这个测试状态之后，
        # 才能用测试数据去测试网络， 否则网络怎么知道啥时候测试啥时候训练？

        # 可视化
        for name, layer in net_prob_0.named_parameters ():
            writer.add_histogram (name + '_grad_normal', layer.grad, epoch)
            writer.add_histogram (name + '_data_normal', layer, epoch)

        for name, layer in net_prob_05.named_parameters ():
            writer.add_histogram (name + '_grad_regularization', layer.grad, epoch)
            writer.add_histogram (name + '_data_regularization', layer, epoch)

        test_pred_prob_0, test_pred_prob_05 = net_prob_0 (test_x), net_prob_05 (test_x)

        # 绘图
        plt.scatter (train_x.data.numpy (), train_y.data.numpy (), c='blue', s=50, alpha=0.3, label='train')
        plt.scatter (test_x.data.numpy (), test_y.data.numpy (), c='red', s=50, alpha=0.3, label='test')
        plt.plot (test_x.data.numpy (), test_pred_prob_0.data.numpy (), 'r-', lw=3, label='d_prob_0')
        plt.plot (test_x.data.numpy (), test_pred_prob_05.data.numpy (), 'b--', lw=3, label='d_prob_05')
        plt.text (-0.25, -1.5, 'd_prob_0 loss={:.8f}'.format (loss_normal.item ()), fontdict={'size': 15, 'color': 'red'})
        plt.text (-0.25, -2, 'd_prob_05 loss={:.6f}'.format (loss_wdecay.item ()), fontdict={'size': 15, 'color': 'red'})

        plt.ylim ((-2.5, 2.5))
        plt.legend (loc='upper left')
        plt.title ("Epoch: {}".format (epoch+1))
        plt.show ()
        plt.close ()
# 这里我们还得告诉网络接下来我们又要进入训练状态了
        net_prob_0.train ()   
        net_prob_05.train ()

标准化之 BN:批标准化， 批指的是 mini-batch, 标准化也就是 0 均值 1 方差
1. 可以用更大学习率，加速模型收敛
2. 可以不用精心设计权值初始化
3. 可以不用 Dropout 或者较小的 Dropout
4. 可以不用 L2 或者较小的 weight decay
5. 可以不用局部响应标准化（AlexNet 中用到过）

In [ ]:
class MLP (nn.Module):
    def __init__(self, neural_num, layers=100):
        super (MLP, self).__init__()
        self.linears = nn.ModuleList ([nn.Linear (neural_num, neural_num, bias=False) for i in range (layers)])
        self.bns = nn.ModuleList ([nn.BatchNorm1d (neural_num) for i in range (layers)])
        self.neural_num = neural_num

    def forward (self, x):

        for (i, linear), bn in zip (enumerate (self.linears), self.bns):
            x = linear (x)
            # x = bn (x)
            x = torch.relu (x)

            if torch.isnan (x.std ()):
                print ("output is nan in {} layers".format (i))
                break

            print ("layers:{}, std:{}".format (i, x.std ().item ()))

        return x

    def initialize (self):
        for m in self.modules ():
            if isinstance (m, nn.Linear):

                # method 1
                # nn.init.normal_(m.weight.data, std=1)    # normal: mean=0, std=1

                # method 2 kaiming
                nn.init.kaiming_normal_(m.weight.data)


neural_nums = 256
layer_nums = 100
batch_size = 16

net = MLP (neural_nums, layer_nums)
# net.initialize ()

inputs = torch.randn ((batch_size, neural_nums))  # normal: mean=0, std=1

output = net (inputs)
print (output)

强大的 BN 依然可以保证数据的尺度，并且好处就是我们不用再考虑用什么样的方式进行权值的初始化

pytorch中的BN：
nn.BatchNorm1d
nn.BatchNorm2d
nn.BatchNorm3d
1. 以上都继承了 __BatchNorm基类：
  1. __init__(self,num_features,eps=1e-5,momentum=0.1,affine=True,track_running_satte=True)
  2. num_features 表示一个样本的特征数量，这是最重要的一个参数
  3. eps 表示分母修正项
  4. omentum 表示指数加权平均估计当前 mean/var。affine 表示是否需要 affine transform, track_running_stats 表示是训练状态还是测试状态
  5. 因为我们发现 momentum 那里有个均值和方差，如果是训练状态，那么就需要重新估计 mean 和方差，而如果是测试状态，就用训练时候统计的均值和方差
2. BatchNorm的三个方法也是有属性的
  2. running_mean: 均值
  3. running_var: 方差
  4. weight: affine transform 中的
  5. bias: affine transforom 中的
  6. 这里的均值和方差是采用指数加权平均进行计算的， 不仅要考虑当前 mini-batch 的均值和方差，还考虑上一个 mini-batch 的均值和方差（当然是在训练的时候，测试的时候是用当前的统计值。）

其他的NormaLization方法，计算均值和标准差都是以特征为单位的
[1,2,3,4]4个参数，
1. Norm1d，则是代表 样本数*特征数*维度，例如一维样本就是长度就是特征数
2. Norm2d，计算是以特征图为单位的，样本数*特征图（卷积核个数）*（2*2）
3. Norm3d，样本数*特征*（2*2*3）

分类：
1. Batch Normalization (BN)
2. Layer Normalization (LN)
3. Instance Normalization (IN)
4. Group Normalization (GN)

以上3中Norm的相同是在公式上面（记得BN的公式，先计算均值，方差，然后利用可学习参数更新）
不同点：在求取μ和σ的方式不同
1. BatchNormalization 我们上面已经学习了，这个是在一个 batch 上去计算均值和方差
2. Layer Normalization 是以层为单位去计算均值和方差
  1. 起因：BN 不适用于变长的网络，如 RNN， 所以提出了逐层计算均值和方差的思路（也就是按照bn计算的方法对每一个特征计算对变长序列不合适）
3.  Instance Normalization 主要在图像生成方法中使用的一个方法
4. Group Normalization 是按组为单位计算均值和方差

In [ ]:
batch_size = 3    # 3 个样本
num_features = 5    # 5 个特征
momentum = 0.3     # 这个计算后面均值，方差的时候用到

features_shape = (1)   # 我们特征的维度是 1

feature_map = torch.ones (features_shape)                                                    # 1D   一个特征
feature_maps = torch.stack ([feature_map*(i+1) for i in range (num_features)], dim=0)         # 2D   一列数据
feature_maps_bs = torch.stack ([feature_maps for i in range (batch_size)], dim=0)             # 3D   上面那 3 列数据
print ("input data:\n {} shape is {}".format (feature_maps_bs, feature_maps_bs.shape))

bn = nn.BatchNorm1d (num_features=num_features, momentum=momentum)   # BN1d 在这里定义

running_mean, running_var = 0, 1   # 这个得初始化，毕竟我们这是第一次迭代

for i in range (2):
  outputs = bn (feature_maps_bs)

  print ("\niteration:{}, running mean: {}".format (i, bn.running_mean))
  print ("iteration:{}, running var:{}".format (i, bn.running_var))

  mean_t, var_t = 2, 0

  running_mean = (1 - momentum) * running_mean + momentum * mean_t         # 采用滑动平均方式计算均值和方差
  running_var = (1 - momentum) * running_var + momentum * var_t

  print ("iteration:{}, 第二个特征的 running mean: {}".format (i, running_mean))
  print ("iteration:{}, 第二个特征的 running var:{}".format (i, running_var))

BN 与 LN 的区别：

1. LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；
2. BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。

在 LN 中不再有 running_mean 和 running_var， 并且 gamma 和 beta 为逐元素的

In [ ]:
# normalized_shape 表示该层特征形状，这个依然是最重要的。eps 表示分母修正项，
#  elementwise_affine 表示是否需要 affine transform。
nn.LayerNorm (normalized_shape, eps=1e-05, elementwise_affine=True)
# 如果我们把参数 elementwise_affine 设置为 False， 会报 AttributeError: 
# 'NoneType' object has no attribute'shape'， 另外还得说一下 normalized_shape，
# 这个我们也可以自己指定， 但得注意一定要从最后维度开始